In [10]:
import numpy as np
from numpy import diff
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import cross_val_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from IPython.display import clear_output
import scipy
print(os.listdir())
%matplotlib inline

['.ipynb_checkpoints', 'cnn.py', 'Comparing methods for predicting height.ipynb', 'correction factors.py', 'curve fit.py', 'curve_fit2.py', 'dnnetwork.py', 'errors.txt', 'interface copy.py', 'interface.py', 'jd_bst.py', 'jd_bst_2.py', 'kmeans_model.pkl', 'my_model.h5', 'my_model.keras', 'nn2.py', 'nnetwork.py', 'PBarnesI.py', 'random_forest.joblib', 'scaler.joblib', 'shaperec.py', 'Similarity comparison (std. dev).ipynb', 'similarity_comparison.py', 'svk_600_60_dt_men_2018.csv', 'svk_600_60_dt_women_2018.csv', 'svk_height_weight_mens_2008_v2.csv', 'svk_height_weight_womens_2018_v2.csv', 'test.html', 'test_model.py']


In [11]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option('display.max_columns', 70)

C:\Users\James\AppData\Local\Temp\ipykernel_12172\4021082537.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [12]:
df_mens = pd.read_csv('svk_height_weight_mens_2008_v2.csv',sep=',', index_col=0).reset_index(drop=True)
df_womens = pd.read_csv('svk_height_weight_womens_2018_v2.csv',sep=',', index_col=0).reset_index(drop=True)

In [13]:
df_mens.sample(2)

,child_id,ATV_8,ATV_9,ATV_10,ATV_11,ATV_12,ATV_13,ATV_14,ATV_15,ATV_16,ATV_17,ATV_18,ATT_8,ATT_9,ATT_10,ATT_11,ATT_12,ATT_13,ATT_14,ATT_15,ATT_16,ATT_17,ATT_18
7482,1000332799,1285.000000,1335.000000,1382.500000,1417.5,1465.000000,1527.000000,1617.000000,1675.0,1685.000000,1695.000000,1725.000000,280.000000,307.500000,325.000000,370.000000,415.000000,453.000000,518.000000,577.5,635.000000,680.000000,690.0
7006,1000315828,1347.916667,1385.833333,1454.916667,1512.0,1553.666667,1656.583333,1743.333333,1782.0,1805.583333,1823.333333,1827.083333,264.166667,314.166667,365.916667,433.166667,464.166667,549.083333,617.166667,631.5,648.583333,657.416667,682.0


In [7]:
len(df_mens)

NameError: name 'df_mens' is not defined

In [11]:
len(df_womens)

9181

In [14]:
atv_column_names = [x for x in df_mens.columns if 'ATV' in x]
df_mens_atv = df_mens[atv_column_names]
atv_column_names = [x for x in df_womens.columns if 'ATV' in x]
df_womens_atv = df_womens[atv_column_names]

In [15]:
df_mens_atv = df_mens_atv[(df_mens_atv != 0).all(1)]

In [14]:
df_womens_atv = df_womens_atv[(df_womens_atv != 0).all(1)]

In [16]:
att_column_names = [x for x in df_mens.columns if 'ATT' in x]
df_mens_att = df_mens[att_column_names]
att_column_names = [x for x in df_womens.columns if 'ATT' in x]
df_womens_att = df_womens[att_column_names]

In [18]:
df_mens_600m = pd.read_csv('svk_600_60_dt_men_2018.csv',sep=',', index_col=0).reset_index(drop=True)
df_womens_600m = pd.read_csv('svk_600_60_dt_women_2018.csv',sep=',', index_col=0).reset_index(drop=True)

In [19]:
att600_column_names = [x for x in df_mens_600m.columns if 'T600' in x]
df_mens_att600 = df_mens_600m[att600_column_names]
att600_column_names = [x for x in df_womens_600m.columns if 'T600' in x]
df_womens_att600 = df_womens_600m[att600_column_names]

In [2]:
atdt_column_names = [x for x in df_mens_600m.columns if 'DT' in x] 
df_mens_atv = df_mens_600m[atdt_column_names]
atdt_column_names = [x for x in df_womens_600m.columns if 'DT' in x]
df_womens_atdt = df_womens_600m[atdt_column_names]

NameError: name 'df_mens_600m' is not defined

In [17]:
print(df_mens_atv)

            ATV_8        ATV_9       ATV_10       ATV_11       ATV_12  \
0     1315.000000  1380.000000  1425.000000  1481.000000  1525.000000   
1     1310.000000  1375.000000  1430.000000  1485.000000  1540.000000   
2     1290.000000  1365.000000  1393.000000  1437.000000  1495.000000   
3     1370.000000  1434.000000  1507.000000  1557.000000  1597.000000   
4     1360.000000  1425.000000  1475.000000  1525.000000  1580.000000   
...           ...          ...          ...          ...          ...   
7572  1249.166667  1298.750000  1345.000000  1405.000000  1468.333333   
7573  1331.666667  1378.333333  1413.333333  1446.666667  1495.000000   
7574  1325.000000  1365.000000  1425.000000  1485.000000  1570.000000   
7575  1245.000000  1304.166667  1354.166667  1406.666667  1488.750000   
7576  1335.000000  1401.666667  1461.666667  1518.666667  1580.666667   

           ATV_13       ATV_14       ATV_15       ATV_16       ATV_17  \
0     1618.000000  1702.000000  1745.000000  1766.

In [18]:
#Because we calculate difefrences between years, we need to name the columns right, so we prepare the names here
atv_column_names_gradient = []
for i in range(8, 18):
    atv_column_names_gradient.append('ATV_diff_'+str(i)+'-'+str(i+1))

# Height men

In [59]:
#Height men
all_all_errors=[]
import numpy as np
for start_age in range(8,18):
    print('------------------')
    print('start_age: ' + str(start_age))
    print('------------------')
    col_index_for_start_age = start_age-8 #first year = 8
    print(col_index_for_start_age)
    diffs = np.diff(df_mens_atv[df_mens_atv.columns[col_index_for_start_age:]].values)
    atv_column_names_gradient_current = atv_column_names_gradient[col_index_for_start_age:]
    df_grad = pd.DataFrame(data=diffs, columns=atv_column_names_gradient_current)

    all_errors=[]
    all_std_devs=[]
    for age in range(10,18):
        print('age:', age)
        col_index_for_age = age-start_age #first year = 8
        remaining_ages_for_difference = 18-age
        number_of_similar_rows_to_include_for_predictions = 100
        predicting_errors =[]
        #Calculate similarities between year differences
        #Simialrities are calculated only for values lower than age
        #print(col_index_for_start_age)
        #print(col_index_for_age)
        #similarity = cosine_similarity(df_mens_atv[df_mens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]])
        similarity = scipy.spatial.distance.cdist(df_mens_atv[df_mens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]], df_mens_atv[df_mens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]], metric='euclidean')
        print(len(similarity))
        #We iterate through rows
        for i in range(len(similarity)):
            row = similarity[i]
            sorted_indexes = list(np.argsort(row)) #We sort values from biggest similarity to lowest-we get indexes of those values
            
           
            del sorted_indexes[sorted_indexes.index(i)] #Remove value of index of the same value to remove similati of the same values

            #We get rows with simmilar differences
            similar_rows = []
            for j in range(number_of_similar_rows_to_include_for_predictions):
                similar_rows.append(df_grad.iloc[sorted_indexes[j]])

            #Now we need to calculate remaining differences to estimate value at 18
            next_diferences = [0]*remaining_ages_for_difference
            #print(similar_rows[-1])
            #print()
            for similar_row in similar_rows:
                for k in range(remaining_ages_for_difference):
                    next_diferences[-k-1] = next_diferences[-k-1] + similar_row[-k-1] #we add difference from simmilar rows for each year
                    #print(similar_row[-k-1])
            next_diferences = np.array(next_diferences)/number_of_similar_rows_to_include_for_predictions #divide to get average
            all_next_differences = sum(next_diferences)
            current_value = 1449.0

            predicted_value = current_value + all_next_differences 
            print(predicted_value/10)
            
            actual_value = df_mens_atv.iloc[i][df_mens_atv.columns[-1]]
            print(actual_value)
            
            predicting_error = abs(predicted_value - actual_value)
            predicting_errors.append(predicting_error)
            print(predicting_error)
            z=z

        #Average predicting error
        average_error = np.median(predicting_errors)
        std_error = np.std(predicting_errors)
        print('Average error: ' + str(average_error))
        print('Std. error: ' + str(std_error))
        all_errors.append(average_error)
        all_std_devs.append(std_error)
    print(all_errors)
    print(all_std_devs)
    print('Sum errors: ' + str(sum(all_errors)))

------------------
start_age: 8
------------------
0
age: 10
7569
181.77552564102564
1774.9999999999995
42.755256410256834


NameError: name 'z' is not defined

# Height women

In [53]:
#Height women
all_all_errors=[]
for start_age in range(8,18):
    print('------------------')
    print('start_age: ' + str(start_age))
    print('------------------')
    col_index_for_start_age = start_age-8 #first year = 8
    diffs = diff(df_womens_atv[df_womens_atv.columns[col_index_for_start_age:]].values)
    atv_column_names_gradient_current = atv_column_names_gradient[col_index_for_start_age:]
    df_grad = pd.DataFrame(data=diffs, columns=atv_column_names_gradient_current)

    all_errors=[]
    all_std_devs=[]
    for age in range(start_age,18):
        #print(age)
        col_index_for_age = age-start_age #first year = 8
        remaining_ages_for_difference = 18-age
        number_of_similar_rows_to_include_for_predictions = 100
        predicting_errors =[]
        #Calculate similarities between year differences
        #Simialrities are calculated only for values lower than age
        #print(col_index_for_start_age)
        #print(col_index_for_age)
        #similarity = cosine_similarity(df_womens_atv[df_womens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]])
        similarity = scipy.spatial.distance.cdist(df_womens_atv[df_womens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]], df_womens_atv[df_womens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]], metric='euclidean')

        #We iterate through rows
        for i in range(len(similarity)):
            row = similarity[i]
            sorted_indexes = list(np.argsort(row)) #We sort values from biggest similarity to lowest-we get indexes of those values
            del sorted_indexes[sorted_indexes.index(i)] #Remove value of index of the same value to remove similati of the same values

            #We get rows with simmilar differences
            similar_rows = []
            for j in range(number_of_similar_rows_to_include_for_predictions):
                similar_rows.append(df_grad.iloc[sorted_indexes[j]])

            #Now we need to calculate remaining differences to estimate value at 18
            next_diferences = [0]*remaining_ages_for_difference
            #print(similar_rows[-1])
            #print()
            for similar_row in similar_rows:
                for k in range(remaining_ages_for_difference):
                    next_diferences[-k-1] = next_diferences[-k-1] + similar_row[-k-1] #we add difference from simmilar rows for each year
                    #print(similar_row[-k-1])
            next_diferences = np.array(next_diferences)/number_of_similar_rows_to_include_for_predictions #divide to get average
            all_next_differences = sum(next_diferences)
            current_value = df_womens_atv.iloc[i][df_womens_atv.columns[col_index_for_start_age + col_index_for_age]]

            predicted_value = current_value + all_next_differences
            actual_value = df_womens_atv.iloc[i][df_womens_atv.columns[-1]]

            predicting_error = abs(predicted_value - actual_value)
            predicting_errors.append(predicting_error)


        #Average predicting error
        average_error = np.median(predicting_errors)
        std_error = np.std(predicting_errors)
        print('Average error: ' + str(average_error))
        print('Std. error: ' + str(std_error))
        all_errors.append(average_error)
        all_std_devs.append(std_error)
    print(all_errors)
    print(all_std_devs)
    print('Sum errors: ' + str(sum(all_errors)))

------------------
start_age: 8
------------------


KeyboardInterrupt: 

## We need std dev for predicting all next years not just at 18 years old. So we modify this a bit

# Height men

In [46]:
#Height men
number_of_similar_rows_to_include_for_predictions = 100
all_all_errors=[]
for start_age in range(8,9): #Predpostavimo da imamo vsa leta za nazaj, ker razlike niso tako velike
    print('------------------')
    print('start_age: ' + str(start_age))
    print('------------------')
    col_index_for_start_age = start_age-8 #first year = 8
    diffs = diff(df_mens_atv[df_mens_atv.columns[col_index_for_start_age:]].values)
    atv_column_names_gradient_current = atv_column_names_gradient[col_index_for_start_age:]
    df_grad = pd.DataFrame(data=diffs, columns=atv_column_names_gradient_current)
    #print(df_grad)
    all_errors=[]
    all_std_errors = []
    
    # for age in range(start_age,18):
    for age in range(9,18):
        print('Data up to age: ' + str(age))
        col_index_for_age = age-start_age # start_age = 8
        
        #remaining_ages_for_difference = 18-age
        
        #Calculate similarities between year differences
        #Simialrities are calculated only for values lower than age
        similarity = scipy.spatial.distance.cdist(df_mens_atv[df_mens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]], df_mens_atv[df_mens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]], metric='euclidean')
        print(len(similarity))
        
        predicting_errors =[]
        all_errors=[]
        all_std_errors = []
        for remaining_ages_for_difference in range(1, 18-age+1): #, 18-age+1):
        #for remaining_ages_for_difference in range(1, 2): #, 18-age+1):
            print(f'Predicting for {remaining_ages_for_difference} years ahead')
            predicting_errors =[]
            #We iterate through rows\
            for i in range(len(similarity)):            
                row = similarity[i]
                #print(df_mens_atv.iloc[i][df_mens_atv.columns[col_index_for_start_age + col_index_for_age]])
                #print(df_mens_atv.iloc[i][df_mens_atv.columns[col_index_for_start_age + col_index_for_age + remaining_ages_for_difference]])
                sorted_indexes = list(np.argsort(row)) #We sort values from biggest similarity to lowest-we get indexes of those values
                
                del sorted_indexes[sorted_indexes.index(i)] #Remove value of index of the same value to remove similati of the same values
                print(len(sorted_indexes))
                
                #We get rows with simmilar differences
                similar_rows = []
                for j in range(number_of_similar_rows_to_include_for_predictions):
                    similar_rows.append(df_grad.iloc[sorted_indexes[j]]) #Original version                    

                print(similar_rows[0:3])
                print('-------------------')
               
                #Now we need to calculate remaining differences to estimate value at x
                next_diferences = [0]*remaining_ages_for_difference
                #print(next_diferences)
                #z=z
                #print()
                for similar_row in similar_rows:
                    for k in range(1, remaining_ages_for_difference+1):
                        #print('k')
                        #print(col_index_for_start_age + col_index_for_age)
                        #print(k-1)
                        #print(similar_row[k-1])
                        #next_diferences[-k-1] = next_diferences[-k-1] + similar_row[-k-1] #we add difference from simmilar rows for each year
                        next_diferences[k-1] = next_diferences[k-1] + similar_row[col_index_for_start_age + col_index_for_age + k-1] #we add difference from simmilar rows for each year
                        #next_diferences[k] = next_diferences[k] + similar_row[k]
                        #print(f'Similar row value: {similar_row[k]}')
                
                next_diferences = np.array(next_diferences)/number_of_similar_rows_to_include_for_predictions #divide to get average
                
                #print('next')
                #print(next_diferences)
                #break
                all_next_differences = sum(next_diferences)
                
                current_value = 1395.0
                print(current_value)
                
                predicted_value = current_value + all_next_differences
                print(predicted_value)
                
                #actual_value = df_mens_atv.iloc[i][df_mens_atv.columns[-1]]
                actual_value = df_mens_atv.iloc[i][df_mens_atv.columns[col_index_for_start_age + col_index_for_age + remaining_ages_for_difference]]
                #print(actual_value)
                z=z
                predicting_error = abs(predicted_value - actual_value)
                predicting_errors.append(predicting_error)


            #Average predicting error
            predicting_errors = [x for x in predicting_errors if x < 150]
            average_error = np.median(predicting_errors)
            std_error = np.std(predicting_errors)
            print('Average error: ' + str(average_error))
            #print(predicting_errors)
            print('Std. error: ' + str(std_error))
            all_errors.append(average_error)
            all_std_errors.append(std_error)
        print(all_errors)
        print(all_std_errors)
        print('Sum errors: ' + str(sum(all_errors)))

------------------
start_age: 8
------------------
Data up to age: 9
7569
Predicting for 1 years ahead
7568
[ATV_diff_8-9       65.0
ATV_diff_9-10      41.0
ATV_diff_10-11     51.0
ATV_diff_11-12     84.0
ATV_diff_12-13     89.0
ATV_diff_13-14    112.0
ATV_diff_14-15     48.0
ATV_diff_15-16     45.0
ATV_diff_16-17     20.0
ATV_diff_17-18      0.0
Name: 3256, dtype: float64, ATV_diff_8-9      65.0
ATV_diff_9-10     50.0
ATV_diff_10-11    54.0
ATV_diff_11-12    26.0
ATV_diff_12-13    74.0
ATV_diff_13-14    75.0
ATV_diff_14-15    55.0
ATV_diff_15-16    36.0
ATV_diff_16-17   -10.0
ATV_diff_17-18    21.0
Name: 688, dtype: float64, ATV_diff_8-9      65.000000
ATV_diff_9-10     62.500000
ATV_diff_10-11    37.500000
ATV_diff_11-12    64.583333
ATV_diff_12-13    60.083333
ATV_diff_13-14    56.666667
ATV_diff_14-15     9.083333
ATV_diff_15-16    11.000000
ATV_diff_16-17    20.833333
ATV_diff_17-18     7.333333
Name: 5690, dtype: float64]
-------------------
1395.0
1450.9788636363637


NameError: name 'z' is not defined

# Height women

In [ ]:
#Height women
number_of_similar_rows_to_include_for_predictions = 100
all_all_errors=[]
for start_age in range(8,9): #Predpostavimo da imamo vsa leta za nazaj, ker razlike niso tako velike
    print('------------------')
    print('start_age: ' + str(start_age))
    print('------------------')
    col_index_for_start_age = start_age-8 #first year = 8
    diffs = diff(df_womens_atv[df_womens_atv.columns[col_index_for_start_age:]].values)
    atv_column_names_gradient_current = atv_column_names_gradient[col_index_for_start_age:]
    df_grad = pd.DataFrame(data=diffs, columns=atv_column_names_gradient_current)

    all_errors=[]
    all_std_errors = []
    for age in range(start_age,18):
        print('Data up to age: ' + str(age))
        col_index_for_age = age-start_age # start_age = 8
        
        #remaining_ages_for_difference = 18-age
        
        #Calculate similarities between year differences
        #Simialrities are calculated only for values lower than age
        similarity = scipy.spatial.distance.cdist(df_womens_atv[df_womens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]], df_womens_atv[df_womens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]], metric='euclidean')

        predicting_errors =[]
        all_errors=[]
        all_std_errors = []
        for remaining_ages_for_difference in range(1, 18-age+1): #, 18-age+1):
        #for remaining_ages_for_difference in range(1, 2): #, 18-age+1):
            print(f'Predicting for {remaining_ages_for_difference} years ahead')
            predicting_errors =[]
            #We iterate through rows
            for i in range(len(similarity)):            
                row = similarity[i]
                #print(df_womens_atv.iloc[i][df_womens_atv.columns[col_index_for_start_age + col_index_for_age]])
                #print(df_womens_atv.iloc[i][df_womens_atv.columns[col_index_for_start_age + col_index_for_age + remaining_ages_for_difference]])
                sorted_indexes = list(np.argsort(row)) #We sort values from biggest similarity to lowest-we get indexes of those values
                del sorted_indexes[sorted_indexes.index(i)] #Remove value of index of the same value to remove similati of the same values

                #We get rows with simmilar differences
                similar_rows = []
                for j in range(number_of_similar_rows_to_include_for_predictions):
                    similar_rows.append(df_grad.iloc[sorted_indexes[j]]) #Original version                    

                #print(similar_rows[0:3])
                #print('-------------------')
                
                #Now we need to calculate remaining differences to estimate value at x
                next_diferences = [0]*remaining_ages_for_difference
                #print(similar_rows[-1])
                #print()
                for similar_row in similar_rows:
                    for k in range(1, remaining_ages_for_difference+1):
                        #print('k')
                        #print(col_index_for_start_age + col_index_for_age)
                        #print(k-1)
                        #print(similar_row[k-1])
                        #next_diferences[-k-1] = next_diferences[-k-1] + similar_row[-k-1] #we add difference from simmilar rows for each year
                        next_diferences[k-1] = next_diferences[k-1] + similar_row[col_index_for_start_age + col_index_for_age + k-1] #we add difference from simmilar rows for each year
                        #next_diferences[k] = next_diferences[k] + similar_row[k]
                        #print(f'Similar row value: {similar_row[k]}')
                next_diferences = np.array(next_diferences)/number_of_similar_rows_to_include_for_predictions #divide to get average
                #print('next')
                #print(next_diferences)
                #break
                all_next_differences = sum(next_diferences)
                current_value = df_womens_atv.iloc[i][df_womens_atv.columns[col_index_for_start_age + col_index_for_age]]

                predicted_value = current_value + all_next_differences
                #actual_value = df_womens_atv.iloc[i][df_womens_atv.columns[-1]]
                actual_value = df_womens_atv.iloc[i][df_womens_atv.columns[col_index_for_start_age + col_index_for_age + remaining_ages_for_difference]]

                predicting_error = abs(predicted_value - actual_value)
                predicting_errors.append(predicting_error)


            #Average predicting error
            predicting_errors = [x for x in predicting_errors if x < 150]
            average_error = np.median(predicting_errors)
            std_error = np.std(predicting_errors)
            print('Average error: ' + str(average_error))
            #print(predicting_errors)
            print('Std. error: ' + str(std_error))
            all_errors.append(average_error)
            all_std_errors.append(std_error)
        print(all_errors)
        print(all_std_errors)
        print('Sum errors: ' + str(sum(all_errors)))